> *This notebook is part of the free course [EEwPython](https://colab.research.google.com/github/csaybar/EEwPython/blob/master/index.ipynb); the content is available [on GitHub](https://github.com/csaybar/EEwPython)* and released under the [Apache 2.0 License](https://www.gnu.org/licenses/gpl-3.0.en.html). 99% of this material has been adapted from [Google Earth Engine Guides](https://developers.google.com/earth-engine/).

<center>
<h1>Google Earth Engine with Python </h1>
<h2> Export Data </h2>
</center>
<h2> Topics:</h2>

1. Exporting Images
2. Exporting Tables and vector data
3. Exporting videos

You can export images, map tiles, tables and video from Earth Engine. The exports can be sent to your Google Drive account, to [Google Cloud Storage](https://cloud.google.com/storage/) or to a new Earth Engine asset.

To use Google Cloud Storage (a fee-based service), you'll need to set up a project, enable billing for the project, and create a storage bucket. See the [Cloud Storage Quickstart](https://cloud.google.com/storage/docs/quickstart-console) page for instructions. See [this guide](https://cloud.google.com/storage/docs/naming) for information on storage bucket naming. Data exported to a Cloud Storage bucket will have the bucket's [default object Access Control List (ACL)](https://cloud.google.com/storage/docs/access-control/create-manage-lists#defaultobjects). You must have write permission for the specified bucket.

The following sections describe each type of export in detail.

## Connecting GEE with Google Services

In [1]:
from google.cloud import storage

## Testing the software setup

In [2]:
# Earth Engine Python API
import ee
ee.Initialize()

In [3]:
import folium

In [4]:
#@title Mapdisplay: Display GEE objects using folium.
def Mapdisplay(center, dicc, Tiles="OpensTreetMap",zoom_start=10):
    '''
    :param center: Center of the map (Latitude and Longitude).
    :param dicc: Earth Engine Geometries or Tiles dictionary
    :param Tiles: Mapbox Bright,Mapbox Control Room,Stamen Terrain,Stamen Toner,stamenwatercolor,cartodbpositron.
    :zoom_start: Initial zoom level for the map.
    :return: A folium.Map object.
    '''
    mapViz = folium.Map(location=center, tiles=Tiles, zoom_start=zoom_start)
    for k, v in dicc.items():
      if ee.image.Image in [type(x) for x in v.values()]:
        tiles = ee.data.getTileUrl(v, 8014, 4817, 37)
        tiles = tiles[:-12] + "{z}/{x}/{y}"
        folium.TileLayer(
            tiles = tiles,
            attr  = 'Google Earth Engine',
            overlay =True,
            name  = k
        ).add_to(mapViz)
      else:
        folium.GeoJson(
        data = v,
        name = k
          ).add_to(mapViz)
    mapViz.add_child(folium.LayerControl())
    return mapViz

# 1. Exporting images

You can export images from Earth Engine in [GeoTIFF](https://trac.osgeo.org/geotiff/) or [TFRecord](https://www.tensorflow.org/api_docs/python/tf/io#TFRecords_Format_Details) format. See [Configuration Parameters](https://developers.google.com/earth-engine/exporting#configuration-parameters) for more output options.


In [5]:
# Load a landsat image and select three bands.
landsat = ee.Image('LANDSAT/LC08/C01/T1_TOA/LC08_123032_20140515')\
            .select(['B4', 'B3', 'B2'])

# Create a geometry representing an export region.
geometry = ee.Geometry.Rectangle([116.2621, 39.8412, 116.4849, 40.01236])
center = geometry.centroid().getInfo()['coordinates']
center.reverse()

Mapdisplay(center,{'landsat':landsat.getMapId()},zoom_start=7)

## to Cloud Storage

To export an image to a Google Cloud Storage bucket, use 
`ee.batch.Export.image.toCloudStorage()`. To export the Landsat image in the previous example to Cloud Storage instead of Drive, use:

In [7]:
outputBucket = 'imagery-bucket'

# Export the image, specifying scale and region.
task = ee.batch.Export.image.toCloudStorage(**{
    'image': landsat,
      'description': 'imageToCOGeoTiffExample',
    'scale': 100,
    'region': geometry.getInfo()['coordinates'],
    'fileFormat': 'GeoTIFF',
    'bucket': outputBucket,
    'formatOptions': {'cloudOptimized': True}
})
task.start()

In [8]:
import time

while task.active():
  print('Polling for task (id: {}).'.format(task.id))
  time.sleep(5)

Polling for task (id: N2S3F5P5ZTLCHGD4U3TCLZHE).
Polling for task (id: N2S3F5P5ZTLCHGD4U3TCLZHE).
Polling for task (id: N2S3F5P5ZTLCHGD4U3TCLZHE).
Polling for task (id: N2S3F5P5ZTLCHGD4U3TCLZHE).
Polling for task (id: N2S3F5P5ZTLCHGD4U3TCLZHE).
Polling for task (id: N2S3F5P5ZTLCHGD4U3TCLZHE).
Polling for task (id: N2S3F5P5ZTLCHGD4U3TCLZHE).
Polling for task (id: N2S3F5P5ZTLCHGD4U3TCLZHE).
Polling for task (id: N2S3F5P5ZTLCHGD4U3TCLZHE).
Polling for task (id: N2S3F5P5ZTLCHGD4U3TCLZHE).
Polling for task (id: N2S3F5P5ZTLCHGD4U3TCLZHE).


We can view the image on GCP's bucket connected that holds the same email we used to create our GEE account.

# 2. Exporting tables and vector data

You can export a `FeatureCollection` as CSV, SHP (shapefile), GeoJSON, KML, KMZ or TFRecord using `Export.table`. The `FeatureCollection` may represent vectors or simply a table of data. In the latter case, the features in the collection will have null geometry.

Note some additional constraints when working with some file formats, including:

- KML: A `FeatureCollection` exported to a KML file will have all the geometries transformed to unprojected (WGS84) coordinates.

- SHP: A `FeatureCollection` exported to a Shapefile must contain features with the same geometry type and projection and must fit within the [Shapefile size limits](https://desktop.arcgis.com/en/arcmap/latest/manage-data/shapefiles/geoprocessing-considerations-for-shapefile-output.htm). Column names are truncated to 10 characters or fewer, and this must not create duplicate column names.

- TFRecord: A `FeatureCollection` exported to a TFRecord file will always be compressed with the GZIP compression type. You always get exactly one TFRecord file. All numbers are coerced to float type.

In [11]:
# Make a collection of points.
features = ee.FeatureCollection([
  ee.Feature(ee.Geometry.Point(30.41, 59.933), {'name': 'Voronoi'}),
  ee.Feature(ee.Geometry.Point(-73.96, 40.781), {'name': 'Thiessen'}),
  ee.Feature(ee.Geometry.Point(6.4806, 50.8012), {'name': 'Dirichlet'})
])
center = features.geometry().centroid().getInfo()['coordinates']
center.reverse()
Mapdisplay(center,{'feature_example':features.getInfo()},zoom_start=2)

## to Cloud Storage

To export a `FeatureCollection` to Cloud Storage, use ee.batch.Export.table.toCloudStorage(). For example, using the features defined previously:



In [12]:
outputBucket = 'imagery-bucket'

# Export a KML file to Cloud Storage.
task = ee.batch.Export.table.toCloudStorage(**{
  'collection': features,
  'description':'vectorsToCloudStorageExample',
  'bucket': outputBucket,
  'fileNamePrefix': 'exampleTableExport',
  'fileFormat': 'KML'    
})

task.start()

In [13]:
while task.active():
  print('Polling for task (id: {}).'.format(task.id))
  time.sleep(5)

Polling for task (id: XC66IZO2FEC2SGRHP3UVE34D).
Polling for task (id: XC66IZO2FEC2SGRHP3UVE34D).
Polling for task (id: XC66IZO2FEC2SGRHP3UVE34D).


There are several limitations on the size and shape of Earth Engine table assets:

- Maximum of 100 million features

- Maximum of 1000 properties (columns)

- Maximum of 100,000 vertices for each row's geometry

- Maximum of 100,000 characters per string value

# 3. Exporting video

To export ordered image collections as video, where frames are defined by images in the collection, use `Export.video()`. You can configure the way the `ImageCollection` is turned into video by setting frame rate, scale and dimensions. The video will be encoded as an MP4.

In [14]:
# Load a Landsat 5 image collection.
# 1. San Francisco Bay.
# 2. Filter cloudy scenes.
# 3. Get 20 years of imagery.
# 4. Need to have 3-band imagery for the video.
# 5. Need to make the data 8-bit.
collection = ee.ImageCollection('LANDSAT/LT05/C01/T1_TOA')\
               .filter(ee.Filter.eq('WRS_PATH', 44))\
               .filter(ee.Filter.eq('WRS_ROW', 34))\
               .filter(ee.Filter.lt('CLOUD_COVER', 30))\
               .filterDate('1991-01-01','2011-12-30')\
               .select(['B4', 'B3', 'B2'])\
               .map(lambda img:img.multiply(512).uint8())

In [15]:
# Define an area to export.
polygon = ee.Geometry.Rectangle([-122.7286, 37.6325, -122.0241, 37.9592]);
center = polygon.centroid().getInfo()['coordinates']
center.reverse()
Mapdisplay(center,{'SF LANDSAT':collection.first().getMapId()})

## to Cloud Storage

To export a video to Cloud Storage, use Export.video.toCloudStorage(). For example, using the ImageCollection from the previous example:

In [17]:
outputBucket = 'imagery-bucket'

# Export video to cloud storage.
task = ee.batch.Export.video.toCloudStorage(**{
    'collection': collection,
    'description': 'sfVideoExampleToCloud',
    'dimensions': 720,
    'bucket': outputBucket,
    'framesPerSecond': 12,
    'region': polygon.getInfo()['coordinates']
})
task.start()

In [18]:
while task.active():
  print('Polling for task (id: {}).'.format(task.id))
  time.sleep(5)

Polling for task (id: YEXVARXAKSUNNYUSIJOHO4GM).
Polling for task (id: YEXVARXAKSUNNYUSIJOHO4GM).
Polling for task (id: YEXVARXAKSUNNYUSIJOHO4GM).
Polling for task (id: YEXVARXAKSUNNYUSIJOHO4GM).
Polling for task (id: YEXVARXAKSUNNYUSIJOHO4GM).
Polling for task (id: YEXVARXAKSUNNYUSIJOHO4GM).
Polling for task (id: YEXVARXAKSUNNYUSIJOHO4GM).
Polling for task (id: YEXVARXAKSUNNYUSIJOHO4GM).
Polling for task (id: YEXVARXAKSUNNYUSIJOHO4GM).
Polling for task (id: YEXVARXAKSUNNYUSIJOHO4GM).
Polling for task (id: YEXVARXAKSUNNYUSIJOHO4GM).
Polling for task (id: YEXVARXAKSUNNYUSIJOHO4GM).
Polling for task (id: YEXVARXAKSUNNYUSIJOHO4GM).
Polling for task (id: YEXVARXAKSUNNYUSIJOHO4GM).
Polling for task (id: YEXVARXAKSUNNYUSIJOHO4GM).
Polling for task (id: YEXVARXAKSUNNYUSIJOHO4GM).
Polling for task (id: YEXVARXAKSUNNYUSIJOHO4GM).
Polling for task (id: YEXVARXAKSUNNYUSIJOHO4GM).
Polling for task (id: YEXVARXAKSUNNYUSIJOHO4GM).
Polling for task (id: YEXVARXAKSUNNYUSIJOHO4GM).
Polling for task (id

---